## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [149]:
path = '../03-corpus/2-data/1-fr/'
acteur = "pinel"

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=',')[["Address", 'Title', 'Type', "text"]]

csv

,Address,Title,Type,text
0,https:////pinel.qc.ca//,Accueil - Institut national de psychiatrie lég...,text//html,L’Institut national de psychiatrie légale Phil...
1,https:////pinel.qc.ca//evenements//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
2,https:////pinel.qc.ca//nouvelles//,Nouvelles - Institut national de psychiatrie l...,text//html,Nouvelles Les journalistes qui désirent réalis...
3,https:////pinel.qc.ca//covid-19//,COVID-19 - Institut national de psychiatrie lé...,text//html,COVID-19 Quelques outils et ressources pour vo...
4,https:////pinel.qc.ca//carriere//,Carrières - Institut national de psychiatrie l...,text//html,Carrières Vous pouvez faire une différence. Co...
...,...,...,...,...
500,https:////pinel.qc.ca//?p=19757,NaN,NaN,Ordres du jour et procès-verbaux 2022 Réunion ...
501,https:////pinel.qc.ca//?p=3192,NaN,NaN,"Auditorium Lionel-Béliveau Le 20 avril 2006, l..."
502,https:////pinel.qc.ca//?p=13292,NaN,NaN,"Professeure adjointe, École de travail social ..."
503,https:////pinel.qc.ca//?p=18296,NaN,NaN,"Professeure agrégée Julie Carpentier, Ph. D. D..."


In [150]:
liste = [lien.replace('//', "/") for lien in csv['Address'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien and not 'pdf' in lien and not 'gaspesie' in lien]
liste 

['https://pinel.qc.ca/',
 'https://pinel.qc.ca/evenements/',
 'https://pinel.qc.ca/covid-19/',
 'https://pinel.qc.ca/carriere/',
 'https://pinel.qc.ca/?s=',
 'https://pinel.qc.ca/soins-aux-patients/',
 'https://pinel.qc.ca/familles-et-visiteurs/',
 'https://pinel.qc.ca/services-aux-professionnels/',
 'https://pinel.qc.ca/centre-de-recherche/',
 'https://pinel.qc.ca/volet-academique/',
 'https://pinel.qc.ca/linstitut/',
 'https://pinel.qc.ca/philosophie-des-soins/',
 'https://pinel.qc.ca/programmes-et-traitements/',
 'https://pinel.qc.ca/aide-et-ressources/',
 'https://pinel.qc.ca/activites-des-patients/',
 'https://pinel.qc.ca/les-installations/',
 'https://pinel.qc.ca/approche-interdisciplinaire/',
 'https://pinel.qc.ca/plaintes-et-qualite-des-services/',
 'https://pinel.qc.ca/programme-quebecois-pour-les-troubles-mentaux/',
 'https://pinel.qc.ca/planifiez-votre-visite/',
 'https://pinel.qc.ca/durant-votre-visite/',
 'https://pinel.qc.ca/comprendre-la-maladie-mentale/',
 'https://pine

In [151]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens = []
for i in range (2, 9): # On prend les liens de niveau 2 à 8
    try:
        liens += [lien for lien in liste if len(lien.split('/')) > i]
    except Exception as e:
        True

In [152]:
regroupements = {}

for lien in liens:
    for i in range(3,9):
        try:
            tag = lien.split('/')[i-1]
            if tag in regroupements:
                regroupements[tag].append(lien)
                
            else:
                if len(tag)>1:
                    regroupements[tag] = []

        except Exception as e:
            True

tags = list(regroupements.keys())

In [153]:
tags = [tag for tag in regroupements if len(regroupements[tag]) > 2]

In [154]:
tags

['pinel.qc.ca',
 'evenements',
 'carriere',
 'philosophie-des-soins',
 'activites-des-patients',
 'stages',
 'events',
 'conference-midi-14',
 'page',
 'career',
 'agente-de-gestion-du-personnel-volet-relations-de-travail-avantages-sociaux-et-remuneration',
 'conseillere-cadre-en-developpement-organisationnel-et-formation',
 'agente-de-planification-de-programmation-et-de-recherche-projet-en-telesante-volet-clinique',
 'agente-de-planification-de-programmation-et-de-recherche-deploiement-de-la-politique-sur-les-mesures-de-controle',
 'agente-administratif-ive-classe-2-liste-de-rappel',
 'agente-de-gestion-du-personnel-volet-dotation-et-formation',
 'coordonnateur-clinico-administratif-en-psychiatrie-legale-programmes-de-traitement-et-de-retablissement',
 'conseiller-cadre-en-mesure-durgence-et-securite-civile',
 'cheffe-dunite-de-soins-et-cheffe-dunite-de-soins-interimaire',
 '7687-2',
 'psychologue',
 'dietetiste-nutritionniste',
 'psychologue-psychiatrie-adulte',
 'criminologue-rempl

In [155]:
print('Ça nous donne {} tags.'.format(len(tags)))
tags

Ça nous donne 54 tags.


['pinel.qc.ca',
 'evenements',
 'carriere',
 'philosophie-des-soins',
 'activites-des-patients',
 'stages',
 'events',
 'conference-midi-14',
 'page',
 'career',
 'agente-de-gestion-du-personnel-volet-relations-de-travail-avantages-sociaux-et-remuneration',
 'conseillere-cadre-en-developpement-organisationnel-et-formation',
 'agente-de-planification-de-programmation-et-de-recherche-projet-en-telesante-volet-clinique',
 'agente-de-planification-de-programmation-et-de-recherche-deploiement-de-la-politique-sur-les-mesures-de-controle',
 'agente-administratif-ive-classe-2-liste-de-rappel',
 'agente-de-gestion-du-personnel-volet-dotation-et-formation',
 'coordonnateur-clinico-administratif-en-psychiatrie-legale-programmes-de-traitement-et-de-retablissement',
 'conseiller-cadre-en-mesure-durgence-et-securite-civile',
 'cheffe-dunite-de-soins-et-cheffe-dunite-de-soins-interimaire',
 '7687-2',
 'psychologue',
 'dietetiste-nutritionniste',
 'psychologue-psychiatrie-adulte',
 'criminologue-rempl

In [156]:
import os, shutil
from pathlib import Path
from slugify import slugify

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags:
    file_name =  acteur + "_" + slugify(str(tag).strip('\n'))

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["Address"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [157]:
with open(folder_path + acteur + '_sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

Après avoir manuellement indexé

In [158]:
from pandas import read_csv
filepath = '../03-corpus/2-sous-corpus/0-semi-automatic_tags_' + acteur + '.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
# csv = csv[csv['oui/non'] != 'x']
# csv = csv[csv['oui/non'] != 'X']
csv = csv[csv['Sous-corpus'] != 'x']
csv = csv[csv['Sous-corpus'] != 'X']

csv

,Regroupement dans l'arborescence du site,Sous-corpus
1,pinel_activites-des-patients.csv,Soins et services
2,pinel_agente-administratif-ive-classe-2-liste-...,Carrières
3,pinel_agente-administrative-classe-1-unite-f-1...,Carrières
4,pinel_agente-de-gestion-du-personnel-volet-dot...,Carrières
5,pinel_agente-de-gestion-du-personnel-volet-rel...,Carrières
6,pinel_agente-de-planification-de-programmation...,Carrières
7,pinel_agente-de-planification-de-programmation...,Carrières
8,pinel_agente-de-planification-de-programmation...,Carrières
9,pinel_analyste-en-informatique-gestion-de-proj...,Carrières
10,pinel_analyste-en-informatique-mandat-lie-la-m...,Carrières


In [159]:
# tags = list(set([tag for tag in csv['oui/non'].tolist()]))
tags = list(set([tag for tag in csv['Sous-corpus'].tolist()]))

tags

['Nouvelles',
 'Soins et services',
 'Carrières',
 'Administration',
 'Pinel',
 'Ressources',
 'Événements']

In [160]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['Sous-corpus'] == tag]
        liste = [str(t).strip('\n') for t in tuples["Regroupement dans l'arborescence du site"].tolist()]
        regroupements[tag] = liste


In [161]:
regroupements

{'Nouvelles': ['pinel_news.csv'],
 'Soins et services': ['pinel_activites-des-patients.csv',
  'pinel_centre-dactivite-physique.csv',
  'pinel_clinique-devaluation-du-risque-de-violence.csv',
  'pinel_clinique-reseau-jeunesse.csv',
  'pinel_nos-programmes-de-telesante.csv',
  'pinel_philosophie-des-soins.csv',
  'pinel_soins-et-services.csv',
  'pinel_unite-de-soins.csv'],
 'Carrières': ['pinel_agente-administratif-ive-classe-2-liste-de-rappel.csv',
  'pinel_agente-administrative-classe-1-unite-f-1.csv',
  'pinel_agente-de-gestion-du-personnel-volet-dotation-et-formation.csv',
  'pinel_agente-de-gestion-du-personnel-volet-relations-de-travail-avantages-sociaux-et-remuneration.csv',
  'pinel_agente-de-planification-de-programmation-et-de-recherche-deploiement-de-la-politique-sur-les-mesures-de-controle.csv',
  'pinel_agente-de-planification-de-programmation-et-de-recherche-projet-en-telesante-volet-clinique.csv',
  'pinel_agente-de-planification-de-programmation-et-de-recherche-volet-ge

In [162]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../03-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    path='../03-corpus/2-sous-corpus/'
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        path='../03-corpus/2-data/1-fr/' + acteur +'/'
        file_path = path + acteur + '_'  + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame['Sous-corpus'] = tag
        big_frame['Corpus'] = acteur
        big_frame = big_frame[["Corpus", "Sous-corpus",	"Address",	"Title",	"Type",	"text"]]
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))

big_frame

Nouvelles	1
Soins et services	16
Carrières	74
Administration	3
Pinel	9
Ressources	1
Événements	11


,Corpus,Sous-corpus,Address,Title,Type,text
0,pinel,Événements,https:////pinel.qc.ca//evenements//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
1,pinel,Événements,https:////pinel.qc.ca//evenements//page//2//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
2,pinel,Événements,https:////pinel.qc.ca//evenements//page//3//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
3,pinel,Événements,https:////pinel.qc.ca//evenements//page//4//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
4,pinel,Événements,https:////pinel.qc.ca//evenement//,"<span class=""text"">Colloques et formations<//s...",NaN,Événements L'Institut national de psychiatrie ...
5,pinel,Événements,https:////pinel.qc.ca//evenement//?filter=conf...,Lien interne,NaN,Événements L'Institut national de psychiatrie ...
6,pinel,Événements,https:////pinel.qc.ca//evenements//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
7,pinel,Événements,https:////pinel.qc.ca//evenements//page//2//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
8,pinel,Événements,https:////pinel.qc.ca//evenements//page//3//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
9,pinel,Événements,https:////pinel.qc.ca//evenements//page//4//,Événements - Institut national de psychiatrie ...,text//html,Événements L'Institut national de psychiatrie ...
